In [2]:
import findspark
findspark.init()
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/07 20:50:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/07 20:50:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/07 20:50:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Boolean column expression
Create a column expressin that will return boolean values

## Example data

In [5]:
mtcars = spark.read.csv('data/SparkData/mtcars.csv', inferSchema=True, header=True)
mtcars = mtcars.withColumnRenamed('_c0', 'model')
mtcars.show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|           Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
|          M

## between() : true/false if the column value is between a given range

In [9]:
# use mpg_between expression to select cyl column values between 4 and 6
mpg_between = mtcars.cyl.between(4,6)
mpg_between

Column<'((cyl >= 4) AND (cyl <= 6))'>

In [10]:
# show cyl column of the first 5 rows with the boolean value of whether the value is in mpg_between expression
mtcars.select(mtcars.cyl, mpg_between).show(5)

+---+---------------------------+
|cyl|((cyl >= 4) AND (cyl <= 6))|
+---+---------------------------+
|  6|                       true|
|  6|                       true|
|  4|                       true|
|  6|                       true|
|  8|                      false|
+---+---------------------------+
only showing top 5 rows



## contains() : true/false if the column value contains a string

In [11]:
# use model_contains expression to select model column value contains 'Ho'
model_contains = mtcars.model.contains('Ho')
model_contains

Column<'contains(model, Ho)'>

In [12]:
# show model column of the first 5 rows with the boolean value of whether the value is in model_contains expression
mtcars.select(mtcars.model, model_contains).show(5)

+-----------------+-------------------+
|            model|contains(model, Ho)|
+-----------------+-------------------+
|        Mazda RX4|              false|
|    Mazda RX4 Wag|              false|
|       Datsun 710|              false|
|   Hornet 4 Drive|               true|
|Hornet Sportabout|               true|
+-----------------+-------------------+
only showing top 5 rows



## endwith() : true/false if the column value ends with a string

In [13]:
# use model_endswith expression to select model column value ends with 't'
model_endswith = mtcars.model.endswith('t')
model_endswith

Column<'endswith(model, t)'>

In [14]:
# show 6 rows of model column value and the boolean value of whether it matches the model_endswith expression without truncate 
mtcars.select(mtcars.model, model_endswith).show(6, truncate=False)

+-----------------+------------------+
|model            |endswith(model, t)|
+-----------------+------------------+
|Mazda RX4        |false             |
|Mazda RX4 Wag    |false             |
|Datsun 710       |false             |
|Hornet 4 Drive   |false             |
|Hornet Sportabout|true              |
|Valiant          |true              |
+-----------------+------------------+
only showing top 6 rows



## isNotNull() : true/false if the column value is not Null

In [15]:
# Create a DataFrame using Row class
from pyspark.sql import Row
df = spark.createDataFrame([Row(name='Tom', height=80), Row(name='Alice', height=None)])
df.show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
|Alice|  NULL|
+-----+------+



In [16]:
# create a height_isNotNull condition to check whether the height column is not null
height_isNotNull = df.height.isNotNull()
height_isNotNull

Column<'(height IS NOT NULL)'>

In [19]:
# show height column and the bollean value of whether it is not Null
df.select(df.height, height_isNotNull).show()

+------+--------------------+
|height|(height IS NOT NULL)|
+------+--------------------+
|    80|                true|
|  NULL|               false|
+------+--------------------+



## isNull() : true/false if the column value is Null

In [20]:
# create a height_isNull condition to check whether the height column is null
height_isNull = df.height.isNull()
height_isNull

Column<'(height IS NULL)'>

In [23]:
# show height column and the bollean value of whether it is Null
df.select(df.height, height_isNull).show()

+------+----------------+
|height|(height IS NULL)|
+------+----------------+
|    80|           false|
|  NULL|            true|
+------+----------------+



## isin() : true/false if the column value is contained by the evaluate argument

In [24]:
# check if carb values are either 2 or 3
carb_isin = mtcars.carb.isin([2, 3])
carb_isin 

Column<'(carb IN (2, 3))'>

In [25]:
# show 10 rows of carb column and the bollean value of whether it is in carb_isin condition
mtcars.select(mtcars.carb, carb_isin).show(10)

+----+----------------+
|carb|(carb IN (2, 3))|
+----+----------------+
|   4|           false|
|   4|           false|
|   1|           false|
|   1|           false|
|   2|            true|
|   1|           false|
|   4|           false|
|   2|            true|
|   2|            true|
|   4|           false|
+----+----------------+
only showing top 10 rows



## like() : true/false if the column value matches a pattern based on a SQL LIKE

In [27]:
# create model_like condition to filter model values start with 'Ho'
model_like = mtcars.model.like('Ho%')
model_like

Column<'model LIKE Ho%'>

In [28]:
# show 10 rows of model column and the bollean value of whether it is in model_like condition
mtcars.select(mtcars.model, model_like).show(10)

+-----------------+--------------+
|            model|model LIKE Ho%|
+-----------------+--------------+
|        Mazda RX4|         false|
|    Mazda RX4 Wag|         false|
|       Datsun 710|         false|
|   Hornet 4 Drive|          true|
|Hornet Sportabout|          true|
|          Valiant|         false|
|       Duster 360|         false|
|        Merc 240D|         false|
|         Merc 230|         false|
|         Merc 280|         false|
+-----------------+--------------+
only showing top 10 rows



## rlike() : true/false if the column value matches a pattern based on a SQL RLike (LIKE with Regex)

In [29]:
# create model_rlike condition to filter model values end with 't'
model_rlike = mtcars.model.rlike('t$')
model_rlike 

Column<'RLIKE(model, t$)'>

In [30]:
# show rows of model column and the bollean value of whether it is in model_rlike condition
mtcars.select(mtcars.model, model_rlike).show()

+-------------------+----------------+
|              model|RLIKE(model, t$)|
+-------------------+----------------+
|          Mazda RX4|           false|
|      Mazda RX4 Wag|           false|
|         Datsun 710|           false|
|     Hornet 4 Drive|           false|
|  Hornet Sportabout|            true|
|            Valiant|            true|
|         Duster 360|           false|
|          Merc 240D|           false|
|           Merc 230|           false|
|           Merc 280|           false|
|          Merc 280C|           false|
|         Merc 450SE|           false|
|         Merc 450SL|           false|
|        Merc 450SLC|           false|
| Cadillac Fleetwood|           false|
|Lincoln Continental|           false|
|  Chrysler Imperial|           false|
|           Fiat 128|           false|
|        Honda Civic|           false|
|     Toyota Corolla|           false|
+-------------------+----------------+
only showing top 20 rows



## startswith(): true/false if the column value starts with a string

In [33]:
# create model_startswithcondition to filter model values start with 'Merc'
model_startswith = mtcars.model.startswith('Merc')
model_startswith 

Column<'startswith(model, Merc)'>

In [34]:
mtcars.select(mtcars.model, model_startswith).show()

+-------------------+-----------------------+
|              model|startswith(model, Merc)|
+-------------------+-----------------------+
|          Mazda RX4|                  false|
|      Mazda RX4 Wag|                  false|
|         Datsun 710|                  false|
|     Hornet 4 Drive|                  false|
|  Hornet Sportabout|                  false|
|            Valiant|                  false|
|         Duster 360|                  false|
|          Merc 240D|                   true|
|           Merc 230|                   true|
|           Merc 280|                   true|
|          Merc 280C|                   true|
|         Merc 450SE|                   true|
|         Merc 450SL|                   true|
|        Merc 450SLC|                   true|
| Cadillac Fleetwood|                  false|
|Lincoln Continental|                  false|
|  Chrysler Imperial|                  false|
|           Fiat 128|                  false|
|        Honda Civic|             